In [1]:
import random
import json
from transformers import BertTokenizer, BertForMaskedLM
from datasets import Dataset
import spacy
import os
import glob
import json
import re

In [2]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
nlp = spacy.load("ru_core_news_lg")

In [3]:
def mask_artistic_modifier(sentence: str):
    """
    Ищет первое ADJ→NOUN или ADV→VERB, подставляет [MASK],
    оборачивает сущность в [TGT]…[/TGT], возвращает dict с полями:
      - text: строка для токенизации (с [MASK] и [TGT] маркерами)
      - target: исходный эпитет/наречие
    """
    doc = nlp(sentence)
    tokens = [tok.text for tok in doc]
    
    for i, tok in enumerate(doc):
        if tok.pos_ == "NOUN" and i > 0 and doc[i-1].pos_ == "ADJ":
            target = doc[i-1].text
            tokens[i-1] = "[MASK]"
            tokens[i]   = "[TGT] " + tokens[i] + " [/TGT]"
            masked = " ".join(tokens)
            masked = re.sub(r'\s+([,.:;!?])', r'\1', masked)
            return {"text": masked, "target": target}
        if tok.pos_ == "VERB" and i > 0 and doc[i-1].pos_ == "ADV":
            target = doc[i-1].text
            tokens[i-1] = "[MASK]"
            tokens[i]   = "[TGT] " + tokens[i] + " [/TGT]"
            masked = " ".join(tokens)
            masked = re.sub(r'\s+([,.:;!?])', r'\1', masked)
            return {"text": masked, "target": target}
    
    return None

In [4]:
corpus = [
    "Но не столько страшен палач, сколько неестественное освещение во сне...",
    "От постели к окну протягивается широкая лунная дорога...",
    "Тогда в потоке складывается непомерной красоты женщина..."
]

for sent in corpus:
    result = mask_artistic_modifier(sent)
    if result:
        # теперь result — это словарь: {"text": ..., "target": ...}
        print(f"Вывод:  {result['text']}")
        print(f"Цель: {result['target']}")
    else:
        print(f"Пропуск: {sent}")
    print("---")

Вывод:  Но не столько [MASK] [TGT] палач [/TGT], сколько неестественное освещение во сне...
Цель: страшен
---
Вывод:  От постели к окну протягивается широкая [MASK] [TGT] дорога [/TGT]...
Цель: лунная
---
Вывод:  Тогда в потоке складывается [MASK] [TGT] красоты [/TGT] женщина...
Цель: непомерной
---


In [5]:
import os, glob, json, re

input_dir = "data/filtered"
output_jsonl = "data/processed/data.jsonl"
os.makedirs(os.path.dirname(output_jsonl), exist_ok=True)

total_files = 0
processed_files = 0
total_lines = 0
processed_lines = 0
error_lines = 0

file_list = glob.glob(f"{input_dir}/*.txt")
total_files = len(file_list)

print("Начала обработки")

with open(output_jsonl, "w", encoding="utf-8") as fout:
    for filepath in file_list:
        processed_files += 1
        with open(filepath, "r", encoding="utf-8") as fin:
            lines = fin.readlines()
            total_lines += len(lines)
            
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                try:
                    entry = mask_artistic_modifier(line)
                    if entry:
                        fout.write(json.dumps(entry, ensure_ascii=False) + "\n")
                        processed_lines += 1
                except Exception as e:
                    error_lines += 1
                    continue

print(f"Файды: {total_files}")
print(f"Строки: {total_lines}")
print(f"Обработаны: {processed_lines}")
print(f"Ошибки: {error_lines}")

Начала обработки
Файды: 108
Строки: 190477
Обработаны: 80198
Ошибки: 0
